In [115]:
import requests
from requests_html import HTML
import pandas as pd 
import time

In [8]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_filter = 'Votes'
url = f'{base_url}{tag}?tab={query_filter}'
print(url)

https://stackoverflow.com/questions/tagged/python?tab=Votes


In [10]:
r = requests.get(url)
html_str = r.text
html = HTML(html=html_str)

In [20]:
# html = HTML(html=html_str)

In [21]:

questions_summaries = html.find('.s-post-summary.js-post-summary') # selenium
questions_classes = html.find('.s-link') # selenium
print(questions_summaries)

[<Element 'div' id='question-summary-231767' class=('s-post-summary', 'js-post-summary') data-post-id='231767' data-post-type-id='1'>, <Element 'div' id='question-summary-419163' class=('s-post-summary', 'js-post-summary') data-post-id='419163' data-post-type-id='1'>, <Element 'div' id='question-summary-394809' class=('s-post-summary', 'js-post-summary') data-post-id='394809' data-post-type-id='1'>, <Element 'div' id='question-summary-100003' class=('s-post-summary', 'js-post-summary') data-post-id='100003' data-post-type-id='1'>, <Element 'div' id='question-summary-82831' class=('s-post-summary', 'js-post-summary') data-post-id='82831' data-post-type-id='1'>, <Element 'div' id='question-summary-38987' class=('s-post-summary', 'js-post-summary') data-post-id='38987' data-post-type-id='1'>, <Element 'div' id='question-summary-89228' class=('s-post-summary', 'js-post-summary') data-post-id='89228' data-post-type-id='1'>, <Element 'div' id='question-summary-273192' class=('s-post-summary'

In [51]:
print(questions_summaries[0].text)
questions_summaries[0].text

12446 votes
51 answers
3.1m views
What does the "yield" keyword do?
What is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...
python
iterator
generator
Alex. S.
140k
asked Oct 23, 2008 at 22:21


'12446 votes\n51 answers\n3.1m views\nWhat does the "yield" keyword do?\nWhat is the use of the yield keyword in Python? What does it do? For example, I\'m trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...\npython\niterator\ngenerator\nAlex. S.\n140k\nasked Oct 23, 2008 at 22:21'

In [47]:
columns = ['votes', 'num_answer', 'views', 'questions', 'short_desc', 'tags','tag1', 'tag2', 'user', 'user_details', 'date']

In [48]:
# this_row = list(questions_summaries[0].text.split('\n'))
# print(this_row)

['12446 votes', '51 answers', '3.1m views', 'What does the "yield" keyword do?', "What is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...", 'python', 'iterator', 'generator', 'Alex. S.', '140k', 'asked Oct 23, 2008 at 22:21']


In [49]:
# len(this_row) == len(columns)

True

In [50]:
# row_data = dict(zip(columns, this_row))
# print(row_data)

{'votes': '12446 votes', 'num_answer': '51 answers', 'views': '3.1m views', 'questions': 'What does the "yield" keyword do?', 'short_desc': "What is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...", 'tags': 'python', 'tag1': 'iterator', 'tag2': 'generator', 'user': 'Alex. S.', 'user_details': '140k', 'date': 'asked Oct 23, 2008 at 22:21'}


In [89]:
key_names = ['question', 'votes', 'tags']
classes_needed = ['.s-link', '.s-post-summary--stats-item.s-post-summary--stats-item__emphasized', '.ml0.list-ls-none.js-post-tag-list-wrapper.d-inline']
this_question_element = questions_summaries[0]
q = this_question_element.find('.s-post-summary--stats-item.s-post-summary--stats-item__emphasized', first=True).text
print(q)
q1 = q.replace(' votes', '')
print(q1)


12446 votes
12446


In [122]:
def clean_scraped_data(text, key_name=None):
    if key_name == 'votes':
        return text.replace(' votes', '')
    if key_name == 'tags':
        return text.replace('\n', ' ')
    return text

In [123]:
datas = []

for q_el in questions_summaries:
    question_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True) 
        # print(sub_el.text)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, key_name=keyname)
    datas.append(question_data)


In [124]:
# datas[0]

In [125]:
def parse_tagged_page(html):
    questions_summaries = html.find('.s-post-summary.js-post-summary')
    key_names = ['question', 'votes', 'tags']
    classes_needed = ['.s-link', '.s-post-summary--stats-item.s-post-summary--stats-item__emphasized', '.ml0.list-ls-none.js-post-tag-list-wrapper.d-inline']
    datas = []
    for q_el in questions_summaries:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True) 
            # print(sub_el.text)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, key_name=keyname)
        datas.append(question_data)
    return datas

In [126]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [127]:
def scrape_tag(tag= 'python', query_filter= 'Votes', max_pages=50, pagesize=15):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        # print(p)
        page_num = p +1
        query_filter = 'Votes'
        url = f'{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}'
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [128]:
datas = scrape_tag(tag='python')

In [129]:
len(datas)

2500

In [130]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,"What does the ""yield"" keyword do?",12447,python iterator generator
1,"What does if __name__ == ""__main__"": do?",7867,python namespaces program-entry-point python-m...
2,Does Python have a ternary conditional operator?,7586,python operators conditional-operator
3,What are metaclasses in Python?,7074,python oop metaclass python-class python-datam...
4,How do I check whether a file exists without e...,6861,python file file-exists


In [131]:
df.shape

(2500, 3)

In [132]:
df.to_csv('python.csv', index=False)